In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage.util import random_noise
from skimage import filters
from skimage.morphology import disk
from skimage import feature
from skimage.filters import laplace
from scipy import signal
from scipy.ndimage import gaussian_laplace
from skimage import img_as_float

#function to display multiple images at once(from Lab4.1)
def imshow_all(*images, titles=None):
    images = [img_as_float(img) for img in images]
  
    if titles is None:
        titles = [''] * len(images)
    vmin = min(map(np.min, images))
    vmax = max(map(np.max, images))
    ncols = len(images)
    height = 6
    width = height * len(images)
    fig, axes = plt.subplots(nrows=1, ncols=ncols,
                             figsize=(width, height))
    for ax, img, label in zip(axes.ravel(), images, titles):
        if(len(img.shape)==3):
            img=np.array(img*255,dtype='uint8')
            ax.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
            ax.set_title(label, fontsize='16')
            ax.axis('off')
        else:
            ax.imshow(img, cmap='gray', vmin=vmin, vmax=vmax)
            ax.set_title(label, fontsize='16')
            ax.axis('off')

def snpamount(x):
  return (x/90)+0.3

def cartoonify(source_img, edge_img):
  copy=source_img.copy()
  copy[edge_img >= 127] = 0 #locate positions of pixels lighter than threshold in edge_img, paint these positions black in source_img
  return copy

#1st Question
img1 = cv2.imread('019.jpg')
img2 = cv2.imread('114.jpg')
resized1 = cv2.resize(img1,(288,512))
resized2 = cv2.resize(img2,(288,512))
im_h = cv2.hconcat([resized1,resized2]) #horizontal concatenation

#2nd Question
gray_img = cv2.cvtColor(im_h, cv2.COLOR_BGR2GRAY)
imshow_all(im_h,gray_img,titles=['Original','Original_gray'])

#3rd Question
#Gauss Noise
gauss_img = random_noise(gray_img,mode='gaussian', seed=4) 
gauss_img = np.array(255*gauss_img,dtype='uint8') #convert to type uint8

#Salt and Pepper Noise
snp_img = random_noise(gray_img,mode='s&p',amount=snpamount(1)) 
snp_img = np.array(255*snp_img,dtype='uint8') #convert to type uint8
imshow_all(gauss_img,snp_img,titles=['Gauss Noise','Salt and Pepper Noise'])

#4th Question
#Gaussian Filter
#Trial 1
gaussian1 = filters.gaussian(gauss_img, 0.5)
gaussian1 = np.array(255*gaussian1,dtype='uint8')
gaussian2 = filters.gaussian(snp_img, 0.5)
gaussian2 = np.array(255*gaussian2,dtype='uint8')
#Trial 2- Choice for Gauss noise
gaussian3 = filters.gaussian(gauss_img, 0.8)
gaussian3 = np.array(255*gaussian3,dtype='uint8')
gaussian4 = filters.gaussian(snp_img, 0.8)
gaussian4 = np.array(255*gaussian4,dtype='uint8')
#Trial 3
gaussian5 = filters.gaussian(gauss_img, 1.5)
gaussian5 = np.array(255*gaussian5,dtype='uint8')
gaussian6 = filters.gaussian(snp_img, 1.5)
gaussian6 = np.array(255*gaussian6,dtype='uint8')
#Preview Gaussian Filter
imshow_all(gaussian1,gaussian2,gaussian3,gaussian4,gaussian5,gaussian6,titles=['Gauss Noise (σ=0.5)', 'Salt and Pepper Noise (σ=0.5)', 'Gauss Noise (σ=0.8)', 'Salt and Pepper Noise (σ=0.8)', 'Gauss Noise (σ=1.5)', 'Salt and Pepper Noise (σ=1.5)'])
#Mean Filter
#Trial 1
neighborhood = disk(1) #area of pixels with radius less than 1 from the origin pixel
mean1 = filters.rank.mean(gauss_img, neighborhood)
mean2 = filters.rank.mean(snp_img, neighborhood)
#Trial 2
neighborhood = disk(3)
mean3 = filters.rank.mean(gauss_img, neighborhood)
mean4 = filters.rank.mean(snp_img, neighborhood)
#Trial 3
neighborhood = disk(5)
mean5 = filters.rank.mean(gauss_img, neighborhood)
mean6 = filters.rank.mean(snp_img, neighborhood)
#Preview Mean Filter
imshow_all(mean1,mean2,mean3,mean4,mean5,mean6,titles=['Gauss Noise (disk(1))', 'Salt and Pepper Noise (disk(1))', 'Gauss Noise (disk(3))', 'Salt and Pepper Noise (disk(3))', 'Gauss Noise (disk(5))', 'Salt and Pepper Noise (disk(5))'])
#Median Filter
#Trial 1
neighborhood = disk(1)
median1 = filters.rank.median(gauss_img, neighborhood)
median2 = filters.rank.median(snp_img, neighborhood)
#Trial 2 - Choice for Salt and Pepper Noise
neighborhood = disk(3)
median3 = filters.rank.median(gauss_img, neighborhood)
median4 = filters.rank.median(snp_img, neighborhood)
#Trial 3
neighborhood = disk(5)
median5 = filters.rank.median(gauss_img, neighborhood)
median6 = filters.rank.median(snp_img, neighborhood)
#Preview Median Filter
imshow_all(median1,median2,median3,median4,median5,median6,titles=['Gauss Noise (disk(1))', 'Salt and Pepper Noise (disk(1))', 'Gauss Noise (disk(3))', 'Salt and Pepper Noise (disk(3))', 'Gauss Noise (disk(5))', 'Salt and Pepper Noise (disk(5))'])

#5th Question
#Canny Filter
#Trial 1
edges1 = feature.canny(gaussian3, sigma=2)
edges2 = feature.canny(median4, sigma=2)
edges1 = np.array(edges1*255, dtype=np.uint8)
edges2 = np.array(edges2*255, dtype=np.uint8)
#Trial 2 - Choice for Salt and Pepper Noise
edges3 = feature.canny(gaussian3, sigma=2.25)
edges4 = feature.canny(median4, sigma=2.25)
edges3 = np.array(edges3*255, dtype=np.uint8)
edges4 = np.array(edges4*255, dtype=np.uint8)
#Trial 3 - Choice for Gaussian Noise
edges5 = feature.canny(gaussian3, sigma=2.5)
edges6 = feature.canny(median4, sigma=2.5)
edges5 = np.array(edges5*255, dtype=np.uint8)
edges6 = np.array(edges6*255, dtype=np.uint8)
#Preview Canny Filter
imshow_all(edges1,edges2,edges3,edges4,edges5,edges6,titles=['Gauss Noise (σ=2)', 'Salt and Pepper Noise (σ=2)', 'Gauss Noise (σ=2.25)', 'Salt and Pepper Noise (σ=2.25)', 'Gauss Noise (σ=2.5)', 'Salt and Pepper Noise (σ=2.5)'])
#Laplacian Filter 
laplacian_kernel = [[0,1,0],[1, -4, 1],[0,1,0]]
#Trial for Gaussian Noise
convimlaplacian1 = np.clip(signal.convolve2d(gaussian3, laplacian_kernel),0,255)  #calculate laplacian
laplacian_filter1 = np.clip(laplace(gaussian3), 0, 1)  #apply filter
#Trial for Salt and Pepper Noise
convimlaplacian2 = np.clip(signal.convolve2d(median4, laplacian_kernel),0,255)
laplacian_filter2 = np.clip(laplace(median4), 0, 1)
#Laplacian Filter 
laplacian_kernel1 = [[-1,-1,-1],[-1, 8, -1],[-1,-1,-1]]
#Trial for Gaussian Noise
convimlaplacian3 = np.clip(signal.convolve2d(gaussian3, laplacian_kernel1),0,255)
laplacian_filter3 = np.clip(laplace(gaussian3), 0, 2)
#Trial for Salt and Pepper Noise
convimlaplacian4 = np.clip(signal.convolve2d(median4, laplacian_kernel1),0,255)
laplacian_filter4 = np.clip(laplace(median4), 0, 2)
imshow_all(laplacian_filter1,laplacian_filter2,laplacian_filter3,laplacian_filter4,titles=['Laplace1_Gauss', 'Laplace1_SnP','Laplace2_Gauss','Laplace2_SnP'])
#Marr-Hildreth Filter
#Trial for Gaussian Noise
#Trial 1
marr_hildreth1 = gaussian_laplace(gaussian3, sigma=2.5)
#Trial 2 
marr_hildreth2 = gaussian_laplace(gaussian3, sigma=2.75)
#Trial 3
marr_hildreth3 = gaussian_laplace(gaussian3, sigma=3)
#Trial for Salt and Pepper Noise
#Trial 1
marr_hildreth4 = gaussian_laplace(median4, sigma=1)
#Trial 2 
marr_hildreth5 = gaussian_laplace(median4, sigma=2)
#Trial 1
marr_hildreth6 = gaussian_laplace(median4, sigma=3)
imshow_all(marr_hildreth1,marr_hildreth2,marr_hildreth3,marr_hildreth4,marr_hildreth5,marr_hildreth6,titles=['Gauss Noise (σ=2.5)','Gauss Noise (σ=2.75)','Gauss Noise (σ=3)','Salt and Pepper Noise (σ=1)]','Salt and Pepper Noise (σ=2)','Salt and Pepper Noise (σ=3)'])

cartoon_gauss_img = cartoonify(im_h, edges5)
cartoon_snp_img = cartoonify(im_h, edges4)
imshow_all(cartoon_gauss_img,cartoon_snp_img,titles=['Cartoon_Gauss','Cartoon_SnP'])
cv2.imshow('Cartoon_Gauss',cartoon_gauss_img)
cv2.waitKey(0)
cv2.imshow('Cartoon_SnP',cartoon_snp_img)
cv2.waitKey(0)
cv2.destroyAllWindows()